In [31]:
import os
import pandas as pd
import json
import requests
import time
from datetime import datetime, timedelta
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [33]:
auth_key = os.getenv('MAIN_API_KEY')

headers = {
    'Authorization': auth_key,
    'Content-Type': 'application/json',
}

In [35]:
params = {
    'dateFrom': '2024-01-29T00:00:00Z',  # Начало отчётного периода в формате RFC3339
    # 'dateTo': '2025-04-20T23:59:59Z',    # Конец отчётного периода в формате RFC3339
    'dateTo': datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),    # Конец отчётного периода в формате RFC3339
    'limit': 100000,          # Максимальное количество строк отчета (integer, optional): По умолчанию: 100000, Максимальное значение: 100000.   
    'rrdid': 0                # Уникальный ID строки отчета (integer, optional): Используется для получения отчета частями. Для первой загрузки: 0. Для последующих вызовов: значение `rrd_id` из последней строки предыдущего вызова.
}

url = f'https://statistics-api.wildberries.ru/api/v5/supplier/reportDetailByPeriod'
all_data = []

while True:
    response = requests.get(url=url, headers=headers, params=params)
    response.raise_for_status()
    
    if not response.json():
        break

    all_data.extend(response.json())

    # Обновляем rrdid для следующего запроса
    params["rrdid"] = response.json()[-1]["rrd_id"]

    # Если данных меньше лимита, значит, мы достигли конца
    if len(response.json()) < params["limit"]:
        break

    # Ждем 60 секунд перед следующим запросом
    time.sleep(60)

df = pd.DataFrame(all_data)
df.tail()

,realizationreport_id,date_from,date_to,create_dt,currency_name,suppliercontract_code,rrd_id,gi_id,dlv_prc,fix_tariff_date_from,fix_tariff_date_to,subject_name,nm_id,brand_name,sa_name,ts_name,barcode,doc_type_name,quantity,retail_price,retail_amount,sale_percent,commission_percent,office_name,supplier_oper_name,order_dt,sale_dt,rr_dt,shk_id,retail_price_withdisc_rub,delivery_amount,return_amount,delivery_rub,gi_box_type_name,product_discount_for_report,supplier_promo,rid,ppvz_spp_prc,ppvz_kvw_prc_base,ppvz_kvw_prc,sup_rating_prc_up,is_kgvp_v2,ppvz_sales_commission,ppvz_for_pay,ppvz_reward,acquiring_fee,acquiring_percent,payment_processing,acquiring_bank,ppvz_vw,ppvz_vw_nds,ppvz_office_name,ppvz_office_id,ppvz_supplier_id,ppvz_supplier_name,ppvz_inn,declaration_number,bonus_type_name,sticker_id,site_country,srv_dbs,penalty,additional_payment,rebill_logistic_cost,storage_fee,deduction,acceptance,assembly_id,srid,report_type,is_legal_entity,trbx_id,installment_cofinancing_amount,wibes_wb_discount_percent,rebill_logistic_org
61865,338812659,2025-04-14,2025-04-20,2025-04-21,RUB,None,2915237844901,0,0.0,,,,0,,,,,Продажа,0,0.0,0.0,0,0.0,,Возмещение за выдачу и возврат товаров на ПВЗ,2025-04-20T00:00:00Z,2025-04-20T00:00:00Z,2025-04-20,24743324756,0.0,0,0,0.0,,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,75.348,0.0,0.0,,,-62.790000,-12.56,,0,0,,,,NaN,0,,False,0.0,0,0.0,0.00,0.0,0.0,0,4762732504155969780.0.0,1,False,,0,0,NaN
61866,338812659,2025-04-14,2025-04-20,2025-04-21,RUB,None,2915237844902,0,0.0,,,,0,,,,,Продажа,0,0.0,0.0,0,0.0,,Возмещение за выдачу и возврат товаров на ПВЗ,2025-04-20T00:00:00Z,2025-04-20T00:00:00Z,2025-04-20,25397265945,0.0,0,0,0.0,,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,99.762,0.0,0.0,,,-83.135000,-16.63,,0,0,,,,NaN,0,,False,0.0,0,0.0,0.00,0.0,0.0,0,df.059fc579d4184beab032e714c8aa21cb.0.0,1,False,,0,0,NaN
61867,338812659,2025-04-14,2025-04-20,2025-04-21,RUB,None,2915237844903,0,0.0,,,,0,,,,,Продажа,0,0.0,0.0,0,0.0,,Возмещение за выдачу и возврат товаров на ПВЗ,2025-04-20T00:00:00Z,2025-04-20T00:00:00Z,2025-04-20,30475592744,0.0,0,0,0.0,,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,114.107,0.0,0.0,,,-95.089167,-19.02,,0,0,,,,NaN,0,,False,0.0,0,0.0,0.00,0.0,0.0,0,5918680195614345345.0.0,1,False,,0,0,NaN
61868,338812659,2025-04-14,2025-04-20,2025-04-21,RUB,None,2915237844904,0,0.0,,,,0,,,,,Продажа,0,0.0,0.0,0,0.0,,Возмещение за выдачу и возврат товаров на ПВЗ,2025-04-20T00:00:00Z,2025-04-20T00:00:00Z,2025-04-20,30475593144,0.0,0,0,0.0,,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,62.446,0.0,0.0,,,-52.038333,-10.41,,0,0,,,,NaN,0,,False,0.0,0,0.0,0.00,0.0,0.0,0,6624486654723676202.0.0,1,False,,0,0,NaN
61869,338812659,2025-04-14,2025-04-20,2025-04-21,RUB,None,2915237844905,0,0.0,,,,0,,,,,,0,0.0,0.0,0,0.0,,Хранение,2025-04-20T00:00:00Z,2025-04-20T00:00:00Z,2025-04-20,0,0.0,0,0,0.0,,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.000,0.0,0.0,,,0.000000,0.00,,0,0,,,,NaN,0,,False,0.0,0,0.0,370.23,0.0,0.0,0,f498b6d67041436e28e69aae6bd129ecdec7e1de,1,False,,0,0,NaN


In [10]:
df['date_from'] = pd.to_datetime(df['date_from'])
df['date_to']   = pd.to_datetime(df['date_to'])

output_dir = Path('finance')
output_dir.mkdir(parents=True, exist_ok=True)

id_region = {}
for (start, end), grp in df.groupby(['date_from', 'date_to']):
    counts = grp['realizationreport_id'].value_counts()
    # ID с максимальным количеством строк — «ru»
    ru_id = counts.idxmax()
    for rid, cnt in counts.items():
        id_region[rid] = 'ru' if rid == ru_id else 'other'

for report_id, region in id_region.items():
    sub = df[df['realizationreport_id'] == report_id]
    start = sub['date_from'].min().strftime('%Y-%m-%d')
    end   = sub['date_to'].max().strftime('%Y-%m-%d')
    filename = f"id_{report_id}_from_{start}_to_{end}_reg_{region}.xlsx"
    sub.to_excel(output_dir / filename, index=False)